In [ ]:
%pip install langchain openai chromadb langchain-community -q

In [86]:
import chromadb

In [39]:
client = chromadb.HttpClient(
    host = "3.35.104.197",
    port = 10090,
)

In [3]:
client.heartbeat()

1742011894606428130

In [59]:
client.list_collections()

['ingredient', 'brand', 'data', 'posts', 'hyqe', 'cosmetic']

In [17]:

# 각 컬렉션의 UUID 가져오기
for name in collections:
    collection = client.get_collection(name)
    print(f"Collection Name: {name}, Access ID: {collection.id}")

In [60]:
collection = client.get_collection("posts")

In [ ]:
# # 메타데이터 기반 검색
result1 = collection.get(
    where={"doc_id": 'f3017cfc-9e94-49bb-af25-4303d6718ca5'}
)
pprint(result1)

In [63]:
len(collection.get()['ids'])

30625

## Tagging chain

In [ ]:
from langchain_chroma import Chroma
from dotenv import load_dotenv
load_dotenv()

COLLECTION_NAME = "collection"
PERSIST_DIRECTORY = "vector_store/collection1"


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_tagging_chain
from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 고객요청의도 추출을 위한 schema 설정
schema = {
    "properties": {
        "scope": {
            "type": "string",
            "enum": ['EXTERNAL',
                    'INTERNAL',
            ]
        }
    }
}
extraction_chain = create_tagging_chain(schema, llm)

In [ ]:
# embedding
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# vector store
vector_store1 = Chroma.from_documents(
    documents=docs1,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME+'1',
    persist_directory=PERSIST_DIRECTORY
)
vector_store2 = Chroma.from_documents(
    documents=docs2, 
    embedding=embedding_model, 
    collection_name=COLLECTION_NAME+'2',
    persist_directory=PERSIST_DIRECTORY
)
vector_store3 = Chroma.from_documents(
    documents=docs3, 
    embedding=embedding_model, 
    collection_name=COLLECTION_NAME+'3',
    persist_directory=PERSIST_DIRECTORY
)

In [ ]:
retriever1 = vector_store1.as_retriever(search_kwargs={"k": 1})
retriever2 = vector_store2.as_retriever(search_kwargs={"k": 1})
retriever3 = vector_store3.as_retriever(search_kwargs={"k": 1})

In [ ]:
retrievers = {
    "": retriever1,
    "EXTERNAL": retriever2,
    "INTERNAL": retriever3,
}

In [ ]:
from langchain_core.runnables import chain

@chain
def custom_chain(question):
    response = extraction_chain.invoke(question)
    #print(response.get('scope', ""))
    retriever = retrievers[response.get('scope') or ""]
    return retriever.invoke(question)

In [ ]:
from langchain.prompts import PromptTemplate

template = """다음과 같은 맥락과 히스토리를 사용하여 질문에 대답하십시오.
맥락: {context}
질문: {question}
도움이 되는 답변:"""

rag_prompt_custom = PromptTemplate.from_template(template)

In [ ]:
def using_intent_get_relevant_document(user_question):
    relevant_document = custom_chain.invoke(user_question)[0]
    return relevant_document

In [ ]:
# RAG chain 설정
from langchain.schema.runnable import RunnablePassthrough

rag_chain = {"context": using_intent_get_relevant_document, "question": RunnablePassthrough()} | rag_prompt_custom | llm

## OpenAI 조회

## retriever

In [4]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o", temperature=0, model_kwargs={"stream": True})

In [8]:
from langchain_openai.embeddings import OpenAIEmbeddings
import chromadb
from langchain_chroma import Chroma
from transformers import AutoTokenizer

In [56]:
retriever = vector_store.as_retriever(search_kwargs={"k":3})

In [5]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_chroma import Chroma
from pprint import pprint

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    직원들에 질문에 아래 문맥을 참고하여 응답을 해주었으면 좋겠어요.
    문맥에는 회사에 대한 내용입니다. 모르는 내용일 경우 모른다고 정확하게 명시해 주세요. 
    
    문맥:\n{context}
    
    
    질문: {question}\n\n답변:
    """
)

def rag_chain(question: str):
    llm = ChatOpenAI(model="gpt-4o",)
        
    docs = retriever.get_relevant_documents(question) 
    context = "\n".join([doc.page_content for doc in docs])
    pprint(docs)
        
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt_template}
    )
    
    return qa_chain.invoke(question)


In [68]:
hyqe = client.get_collection("hyqe")
posts = client.get_collection("posts")

In [85]:
posts.get()["metadatas"][0]

{'authority_level': 0,
 'company_id': 2,
 'doc_id': 'd22e568b-37ac-4bdc-83b2-c2b36c15db4b',
 'document_id': 122,
 'post_ctgry': '경영실적',
 'post_id': 10,
 'post_type': 'REPORT',
 'scope': 'INNER',
 'title': '아모레퍼시픽 2012년 1/4분기 경영실적'}

In [64]:
result = rag_chain('설화수 설립연도 알려줘')

[Document(metadata={'brand_id': 1, 'company_id': 2, 'doc_id': 'a30e85a8-23bf-4ecb-adae-8cd7145266c6', 'scope': 'INNER'}, page_content='브랜드 설명: 설화수는 아모레퍼시픽의 대표 럭셔리 브랜드로, 한방 원료와 현대 과학을 결합하여 피부에 균형과 활력을 주는 제품들을 제공합니다. 1997년 설립된 이후, 설화수는 한국의 전통적인 미의 철학을 글로벌 시장에 알려왔습니다. 주요 제품으로는 자음생크림과 윤조에센스 등이 있으며, 특히 인삼과 같은 한방 성분을 활용한 스킨케어 라인은 고급스럽고 차별화된 경험을 제공합니다. ,\n영어 이름: sulwhasoo,\n한글 이름: 설화수,\n브랜드 subtitle: 예술과 헤리티지로 빚어내는 아름다움의 세계,\n영어 이름: sulwhasoo,\nCEO: william.김,\n설립연도: 1997'),
 Document(metadata={'brand_id': 14, 'company_id': 2, 'doc_id': 'c728d799-94f5-4441-a45a-255f31f1b886', 'scope': 'INNER'}, page_content="브랜드 설명: 미쟝센은 '아름다움의 완성'이라는 의미를 지니고 있으며, 다양한 헤어 타입과 스타일에 맞춘 혁신적인 제품을 제공합니다. 특히, 손상 모발을 위한 케어 라인과 스타일링 제품이 많은 사랑을 받고 있습니다. 품질과 트렌드를 선도하는 미쟝센은 소비자들에게 아름답고 건강한 헤어 케어 솔루션을 제공하며, 국내외 시장에서 높은 인지도를 자랑합니다.,\n영어 이름: mise en scene,\n한글 이름: 미쟝센,\n브랜드 subtitle: CHANGE YOUR HAIR, CHANGE YOUR LIFE,\n영어 이름: mise en scene,\nCEO: 김승환,\n설립연도: 2000"),
 Document(metadata={'brand_id': 26, 'company_id': 1, 'doc

In [65]:
result

{'query': '설화수 설립연도 알려줘', 'result': '설화수는 1997년에 설립되었습니다.'}

In [ ]:
from langchain_core.tools import tool
from langchain.prompts import PromptTemplate
from langchain_openai.embeddings import OpenAIEmbeddings
import chromadb

from pprint import pprint
import chromadb

In [ ]:
LLM_MODEL = "gpt-4o"
COLLECTION_NAME_POST = "posts"
COLLECTION_NAME_BRAND = "brand"
COLLECTION_NAME_COSMETIC = "cosmetic"
COLLECTION_NAME_INGREDIENT = "ingredient"
COLLECTION_NAME_LAW = "law"

client = chromadb.HttpClient(host="3.35.104.197", port=10090)
llm = ChatOpenAI(model_name="gpt-4",)

embedding_function = OpenAIEmbeddings(
    base_url = "https://76mqtyy2wie64y-10050.proxy.runpod.net/v1",
    model = "multilingual-e5-large",
    api_key="team3",
    tiktoken_enabled=False,
    embedding_ctx_length = 502
)

collections = {
    "post": client.get_collection(COLLECTION_NAME_POST),
    "brand": client.get_collection(COLLECTION_NAME_BRAND),
    "cosmetic": client.get_collection(COLLECTION_NAME_COSMETIC),
    "ingredient": client.get_collection(COLLECTION_NAME_INGREDIENT),
    # "law": client.get_collection(COLLECTION_NAME_LAW),
}


In [ ]:
@tool
def  retrieve_cosmetics(q: str) -> dict:
    """ 화장품을 조회해오는 함수이다. 
    자사브랜드 : 
    
    의 화장품을 조회해와야하면 
    """
    
    is_amore = True
    return {"is_amore": is_amore, "query" : "쿼리내용"}


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


In [ ]:

# rag_prompt_template = PromptTemplate(
#     template="""당신은 아모레퍼시픽의 전문가입니다.
# 질문: {question}

# 다음의 관련 문서를 참고하여 답변을 생성하세요.
# {context}

# 답변:""",
#     input_variables=["question", "context"],
# )

# def multi_collection_retriever(query, k=5):
#     """
#     여러 개의 Collection에서 데이터를 검색 (query를 벡터화하여 검색)
#     """
#     retrieved_docs = {name: [] for name in collections.keys()}

#     query_embedding = embedding_function.embed_query(query)

#     for collection_name, collection in collections.items():
#         results = collection.query(
#             query_embeddings=[query_embedding],
#             n_results=k
#         )
#         for i in range(len(results['ids'][0])): 
#             retrieved_docs[collection_name].append({
#                 "content": results['documents'][0][i],
#                 "metadata": results['metadatas'][0][i],
#                 "score": results['distances'][0][i]
#             })

#     return retrieved_docs 

# def query_rag_chain(question: str):
#     retrieved_docs = multi_collection_retriever(question, k=5)
    
#     if all(len(docs) == 0 for docs in retrieved_docs.values()):
#         return {
#             "question": question,
#             "response": "관련 정보를 찾을 수 없습니다.",
#             "references": []
#         }
    
#     context_sections = []
#     references = []

#     for collection_name, docs in retrieved_docs.items():
#         if docs:
#             doc_texts = "\n".join([doc["content"] for doc in docs])
#             context_sections.append(f"### {collection_name.upper()} 관련 정보 ###\n{doc_texts}")
#             references.extend([doc["metadata"] for doc in docs])
    


#     context = "\n\n".join(context_sections)[:7500]

#     response = llm.invoke(rag_prompt_template.format(question=question, context=context))

#     return {
#         "question": question,
#         "response": response.content,
#         "references": references
#     }

In [32]:
response = query_rag_chain('설화수에서 미백제품 소개해줘')

In [35]:
print(response.get('response'))

설화수의 미백 제품으로는 '자정미백에센스'와 '블루미너스 팩트'가 있습니다. 

'자정미백에센스'는 백삼의 힘으로 피부 속을 탄탄하게 채우고 맑고 투명한 빛으로 채워주는 제품으로, 칙칙함, 잡티, 불균일한 피부톤 등 복합적인 미백 고민을 해결해줍니다. 한층 더 강력해진 백삼 미백 성분이 피부에 그늘이 드리운 듯한 칙칙함과 잡티를 개선해주며, 기존 대비 2.5배 강화된 백삼농축다당체 성분으로 피부 속을 촉촉하게 적셔 맑은 물을 머금은 듯 투명하게 빛나는 피부를 선사합니다.

'블루미너스 팩트'는 블루 부스팅 파우더의 빛 케어 효과로 피부에 투명하고 화사한 젊음의 빛을 선사하는 제품입니다. 블루 빛의 반사율이 높을수록 피부가 투명해 보이는 것에 주목하여 블루 빛만을 반사시키는 블루 부스팅 파우더를 함유하고 있으며, 설화수의 대표 보습 원료인 맥문동 추출물을 함유하여 끈적임 없이 촉촉한 마무리감을 선사해줍니다. 

두 제품 모두 피부에 활력을 불어넣고, 미백 기능성 성분을 함유하여 피부를 맑고 투명하게 가꾸어 주는 효과가 있습니다.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.tools import tool
from langchain.schema import SystemMessage
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
import chromadb
import json

def get_fresh_llm():
    """매번 새로운 LLM 인스턴스를 생성하여 초기화"""
    return ChatOpenAI(model_name="gpt-4", temperature=0, cache=False)

embedding_function = OpenAIEmbeddings(
    base_url="https://76mqtyy2wie64y-10050.proxy.runpod.net/v1",
    model="multilingual-e5-large",
    api_key="team3",
    tiktoken_enabled=False,
    embedding_ctx_length=502
)

client = chromadb.HttpClient(host="3.35.104.197", port=10090)

classification_prompt = PromptTemplate(
    template="""당신은 아모레퍼시픽 전문가입니다. 다음 질문을 적절한 카테고리로 분류하세요.
    
    아모레퍼시픽의 자사브랜드: 
    카테고리: BEAUTY_CARE, 브랜드명(한글):설화수, 브랜드명(영어) sulwhasoo
    카테고리: BEAUTY_CARE, 브랜드명(한글):라네즈, 브랜드명(영어) laneige
    카테고리: BEAUTY_CARE, 브랜드명(한글):이니스프리, 브랜드명(영어) innisfree
    카테고리: BEAUTY_CARE, 브랜드명(한글):에이피뷰티, 브랜드명(영어) ap
    카테고리: BEAUTY_CARE, 브랜드명(한글):헤라, 브랜드명(영어) hera
    카테고리: BEAUTY_CARE, 브랜드명(한글):프리메라, 브랜드명(영어) primera
    카테고리: BEAUTY_CARE, 브랜드명(한글):아이오페, 브랜드명(영어) iope
    카테고리: BEAUTY_CARE, 브랜드명(한글):마몽드, 브랜드명(영어) mamonde
    카테고리: BEAUTY_CARE, 브랜드명(한글):한율, 브랜드명(영어) hanyul
    카테고리: MEDICAL_BEAUTY, 브랜드명(한글):에스트라, 브랜드명(영어) aestura
    카테고리: BEAUTY_CARE, 브랜드명(한글):에스쁘아, 브랜드명(영어) espoir
    카테고리: BEAUTY_CARE, 브랜드명(한글):에뛰드, 브랜드명(영어) etude
    카테고리: HAIR, 브랜드명(한글):려, 브랜드명(영어) ryo
    카테고리: HAIR, 브랜드명(한글):미쟝센, 브랜드명(영어) mise en scene
    카테고리: HAIR, 브랜드명(한글):라보에이치, 브랜드명(영어) laboh
    카테고리: HAIR, 브랜드명(한글):아윤채, 브랜드명(영어) ayunche
    카테고리: HAIR, 브랜드명(한글):아모스프로페셔널, 브랜드명(영어) amos professional
    카테고리: HAIR, 브랜드명(한글):롱테이크, 브랜드명(영어) longtake
    카테고리: BODY, 브랜드명(한글):일리윤, 브랜드명(영어) illiyoon
    카테고리: BODY, 브랜드명(한글):해피바스, 브랜드명(영어) happybath
    카테고리: BODY, 브랜드명(한글):스킨유, 브랜드명(영어) skin u
    카테고리: ORAL_CARE, 브랜드명(한글):메디안, 브랜드명(영어) median
    카테고리: ORAL_CARE, 브랜드명(한글):젠티스트, 브랜드명(영어) gentist
    카테고리: PERFUME, 브랜드명(한글):구딸, 브랜드명(영어) goutal
    카테고리: INNER_BEAUTY, 브랜드명(한글):바이탈뷰티, 브랜드명(영어) vital beauty
    카테고리: TEA_CULTURE, 브랜드명(한글):오설록, 브랜드명(영어) osulloc
    카테고리: BEAUTY_DEVICE, 브랜드명(한글):메이크온, 브랜드명(영어) makeon
    카테고리: BEAUTY_CARE, 브랜드명(한글):오딧세이, 브랜드명(영어) odyssey
    카테고리: BEAUTY_CARE, 브랜드명(한글):비레디, 브랜드명(영어) bready
    카테고리: BEAUTY_CARE, 브랜드명(한글):홀리추얼, 브랜드명(영어) holitual

    질문: "{question}"
    
    가능한 카테고리:
    - "ingredient" (성분, 화장품 기능 관련 질문)
    - "amore_cosmetic" (자사 화장품 검색 질문)
    - "other_cosmetic" (타사 화장품 검색 질문)
    - "cosmetic" (브랜드 구별 없이 화장품에 관한 질문질문)
    - "company" (회사 전반적인 질문)
    - "news" (최신 뉴스나 기사 관련 질문)

    반드시 위의 카테고리 중 하나만 출력하세요.
    
    자사 
    카테고리:""",
    input_variables=["question"],
)

@tool
def classify_question(question: str) -> str:
    """질문을 적절한 카테고리로 분류합니다."""
    llm = get_fresh_llm() 
    response = llm.invoke(classification_prompt.format(question=question))
    return response.content.strip().replace('"', '')

@tool
def multi_collection_retriever(input_data: str) -> str:
    """
    사용자의 질문을 기반으로 적절한 ChromaDB 컬렉션에서 검색 후 JSON 형식의 결과를 반환합니다.
    input_data는 JSON 문자열이며, 'query'와 'category'를 포함해야 합니다.
    """
    data = json.loads(input_data) 
    query = data["query"]
    category = data["category"]  

    retrieved_docs = {} 
    query_embedding = embedding_function.embed_query(query)

    print(f"검색할 카테고리: {category}")

    search_collections = []
    metadata_filter = None 

    if category == "ingredient":
        search_collections = ["ingredient", "cosmetic"]
    elif category == "amore_cosmetic":
        search_collections = ["cosmetic"]
        metadata_filter = {"scope": "자사"}
    elif category == "other_cosmetic":
        search_collections = ["cosmetic"]
        metadata_filter = {"scope": "타사"} 
    elif category == "cosmetic":
        search_collections = ["cosmetic", "ingredient"] 
    elif category == "company":
        search_collections = ["brand", "posts"]
    elif category == "news":
        search_collections = ["posts"]

    print(f"검색할 컬렉션: {search_collections}")
    print(f"적용할 메타데이터 필터: {metadata_filter}")

    for collection_name in search_collections:
        if metadata_filter:
            results = collections[collection_name].query(
                query_embeddings=[query_embedding], 
                n_results=5,
                where=metadata_filter 
            )
        else:
            results = collections[collection_name].query(
                query_embeddings=[query_embedding], 
                n_results=5
            )

        retrieved_docs[collection_name] = [
            {"content": results['documents'][0][i], "metadata": results['metadatas'][0][i]}
            for i in range(len(results['ids'][0]))
        ]

    return json.dumps(retrieved_docs, ensure_ascii=False)

response_prompt = PromptTemplate(
    template="""당신은 아모레퍼시픽 전문가입니다.

    질문: {question}

    아래 정보를 참고하여 답변을 생성하세요.

    {context}

    답변:""",
    input_variables=["question", "context"],
)

@tool
def generate_response(input_data: str) -> str:
    """
    검색된 데이터를 바탕으로 LLM이 최종 응답을 생성합니다.
    input_data는 JSON 형식으로 전달되며, 'question'과 'retrieved_docs'를 포함합니다.
    """
    llm = get_fresh_llm() 
    data = json.loads(input_data)
    question = data["question"]
    retrieved_docs = data["retrieved_docs"]

    context_sections = []
    for collection_name, docs in retrieved_docs.items():
        if docs:
            doc_texts = "\n".join([doc["content"] for doc in docs])
            context_sections.append(f"### {collection_name.upper()} 관련 정보 ###\n{doc_texts}")

    context = "\n\n".join(context_sections)[:7000]

    response = llm.invoke(response_prompt.format(question=question, context=context))
    return response.content.strip()

tools = [classify_question, multi_collection_retriever, generate_response]

agent = initialize_agent(
    tools=tools,
    llm=get_fresh_llm(), 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

def agentic_rag_chain(question: str):
    category = classify_question(question) 
    retrieved_docs = multi_collection_retriever(json.dumps({"query": question, "category": category}, ensure_ascii=False)) 
    # response = generate_response(json.dumps({"question": question, "retrieved_docs": json.loads(retrieved_docs)}, ensure_ascii=False))

    return {
        "question": question,
        "category": category,
        # "response": response,
        "retrieved_docs": json.loads(retrieved_docs)
    }


In [111]:
query1 = "아모레퍼시픽 미백제품에 대해서 알려줘"
response1 = agentic_rag_chain(query1)


검색할 카테고리: amore_cosmetic
검색할 컬렉션: ['cosmetic']
적용할 메타데이터 필터: {'scope': '자사'}


In [109]:
pprint(response1)

{'category': 'cosmetic',
 'question': '아모레퍼시픽 미백제품에 대해서 알려줘',
 'retrieved_docs': {'cosmetic': [{'content': 'cosmetic_id: 2021,\n'
                                             'category_1: 메이크업,\n'
                                             'category_2: 베이스,\n'
                                             '제품이름: 퍼펙팅/에어리 쿠션 15g,\n'
                                             '브랜드: 설화수,\n'
                                             '제품정보: 없음,\n'
                                             '가격: 44500\n'
                                             '용도: 15g,\n'
                                             '사양: 모든피부\n'
                                             '기한: None\n'
                                             '사용법: 기초 사용 후 파운데이션 사용 단계에서 '
                                             '퍼프를이용하여 적당량을 취한 후 얼굴에 두드리듯이 '
                                             '발라줍니다.\n'
                                             '성분: 정제수, 티타늄디옥사이드 (CI 77891), '
                              

In [83]:

# === 8. 실행 예제 ===
query1 = "미백제품에 대해서 알려줘"
query2 = "설화수 순행클렌징폼과 비슷한 성분을 가진 제품을 알려줘"
query3 = "설화수 순행클렌징폼 성분에 대해 알려줘"

response1 = agentic_rag_chain(query1)
response2 = agentic_rag_chain(query2)
response3 = agentic_rag_chain(query3)


In [84]:
pprint(response1)

{'category': '"cosmetic"', 'question': '미백제품에 대해서 알려줘', 'retrieved_docs': {}}


In [80]:
pprint(response2)

{'category': '"cosmetic"',
 'question': '설화수 순행클렌징폼과 비슷한 성분을 가진 제품을 알려줘',
 'response': "설화수 순행 클렌징폼과 비슷한 성분을 가진 제품으로는 아모레퍼시픽의 '일리윤 세라마이드 아토 수딩 젤'이 있습니다. "
             '이 제품은 설화수 클렌징폼과 같이 세라마이드 성분이 함유되어 있어 피부를 부드럽게 클렌징하면서도 수분을 잃지 않게 '
             '해줍니다. 또한, 민감한 피부에도 사용할 수 있도록 저자극 성분을 사용하였습니다.',
 'retrieved_docs': {}}


In [81]:

pprint(response3)

{'category': '"ingredient"',
 'question': '설화수 순행클렌징폼 성분에 대해 알려줘',
 'response': '설화수 순행클렌징폼은 다양한 천연 성분들로 구성되어 있습니다. 주요 성분으로는 설탕, 글리세린, 스테아릭애씨드, '
             '라우릭애씨드, 팔미틱애씨드, 포타슘하이드록사이드, 마이리스틱애씨드 등이 있습니다. 이 외에도 다양한 식물성 성분들이 '
             '함유되어 있어 피부에 부드럽게 작용하며, 피부를 깨끗하게 클렌징해줍니다.',
 'retrieved_docs': {}}
